## Initial Feature Engineering 1

In [1]:
import numpy as np
import pandas as pd

order_new = pd.read_csv("E:\Sebnewrepo\Data\order_new.csv")
order_product_new = pd.read_csv("E:\Sebnewrepo\Data\order_product_new.csv")
order_new.drop('Unnamed: 0', axis = 1, inplace = True)
order_new.drop('ranking', axis = 1, inplace = True)
order_new.drop('count', axis = 1, inplace = True)

<bound method NDFrame.describe of          order_id  user_id  order_number  order_dow  order_hour_of_day  \
0         2539329        1             1          2                  8   
1         2398795        1             2          3                  7   
2          473747        1             3          3                 12   
3         2254736        1             4          4                  7   
4          431534        1             5          4                 15   
...           ...      ...           ...        ...                ...   
3421078   2266710   206209            10          5                 18   
3421079   1854736   206209            11          4                 10   
3421080    626363   206209            12          1                 12   
3421081   2977660   206209            13          1                 12   
3421082    272231   206209            14          6                 14   

         days_since_prior_order eval_set  
0                           NaN   

In [2]:
order_new = order_new.fillna(0)

In [3]:
# join orders_new and order_product_new together
join = pd.merge(order_product_new, order_new, on = 'order_id', how = 'left' )
join.head()

,order_id,product_id,add_to_cart_order,reordered,user_id,order_number,order_dow,order_hour_of_day,days_since_prior_order,eval_set
0,2,33120,1,1,202279,3,5,9,8.0,prior
1,2,28985,2,1,202279,3,5,9,8.0,prior
2,2,9327,3,0,202279,3,5,9,8.0,prior
3,2,45918,4,1,202279,3,5,9,8.0,prior
4,2,30035,5,0,202279,3,5,9,8.0,prior


#### 1. order_product_prior

In [4]:
order_product_prior = join[join['eval_set'] == 'prior']
order_product_prior.shape

(32434489, 10)

In [5]:
order_product_prior.head()

,order_id,product_id,add_to_cart_order,reordered,user_id,order_number,order_dow,order_hour_of_day,days_since_prior_order,eval_set
0,2,33120,1,1,202279,3,5,9,8.0,prior
1,2,28985,2,1,202279,3,5,9,8.0,prior
2,2,9327,3,0,202279,3,5,9,8.0,prior
3,2,45918,4,1,202279,3,5,9,8.0,prior
4,2,30035,5,0,202279,3,5,9,8.0,prior


#### 2. user_feature_1

In [6]:
df1 = order_new[order_new['eval_set'] == 'prior']

user_id = df1['user_id'].unique()
user_orders = df1.groupby(['user_id'])['order_number'].max()
sum_days_since_prior_order = df1.groupby(['user_id'])['days_since_prior_order'].sum()
avg_days_since_prior_order = df1.groupby(['user_id'])['days_since_prior_order'].mean()

In [7]:
user_feature_1 = pd.DataFrame({
    'user_orders': user_orders,
    'sum_days_since_prior_order': sum_days_since_prior_order,
    'avg_days_since_prior_order': avg_days_since_prior_order
})
user_feature_1 = user_feature_1.reset_index()
user_feature_1.head(10)

,user_id,user_orders,sum_days_since_prior_order,avg_days_since_prior_order
0,1,10,176.0,17.600000
1,2,14,198.0,14.142857
2,3,12,133.0,11.083333
3,4,5,55.0,11.000000
4,5,4,40.0,10.000000
5,6,3,18.0,6.000000
6,7,20,203.0,10.150000
7,8,3,60.0,20.000000
8,9,3,36.0,12.000000
9,10,5,79.0,15.800000


#### 03. user_feature_2

In [8]:
df = order_product_prior[order_product_prior['order_number'] > 1]

total_products = order_product_prior.groupby('user_id')['product_id'].count()
distinct_products = order_product_prior.groupby('user_id')['product_id'].nunique()
reordered_sum = order_product_prior.groupby('user_id')['reordered'].sum()
reordered_count = df.groupby('user_id')['reordered'].count()

In [9]:
user_feature_2 = pd.DataFrame({
    'total_products': total_products,
    'distinct_products': distinct_products,
    'reordered_ratio': reordered_sum/reordered_count
})
user_feature_2.head(10)

,total_products,distinct_products,reordered_ratio
user_id,,,
1,59,18,0.759259
2,195,102,0.510989
3,88,33,0.705128
4,18,17,0.071429
5,37,23,0.538462
6,14,12,0.200000
7,206,68,0.711340
8,49,36,0.464286
9,76,58,0.391304


In [10]:
user_feature_2 = user_feature_2.reset_index()
user_feature_2.head(10)

,user_id,total_products,distinct_products,reordered_ratio
0,1,59,18,0.759259
1,2,195,102,0.510989
2,3,88,33,0.705128
3,4,18,17,0.071429
4,5,37,23,0.538462
5,6,14,12,0.200000
6,7,206,68,0.711340
7,8,49,36,0.464286
8,9,76,58,0.391304
9,10,143,94,0.355072


#### 04. up_feature

In [11]:
m = order_product_prior.groupby(['user_id','product_id'])

In [12]:
up_feature = pd.DataFrame({
    'up_orders': m['order_number'].count(),
    'up_first_order': m['order_number'].min(),
    'up_last_order': m['order_number'].max(),
    'up_average_cart_position': m['add_to_cart_order'].mean()
})

In [13]:
up_feature = up_feature.reset_index()
up_feature.head(10)

,user_id,product_id,up_orders,up_first_order,up_last_order,up_average_cart_position
0,1,196,10,1,10,1.400000
1,1,10258,9,2,10,3.333333
2,1,10326,1,5,5,5.000000
3,1,12427,10,1,10,3.300000
4,1,13032,3,2,10,6.333333
5,1,13176,2,2,5,6.000000
6,1,14084,1,1,1,2.000000
7,1,17122,1,5,5,6.000000
8,1,25133,8,3,10,4.000000
9,1,26088,2,1,2,4.500000


#### 5. prd_feature

In [14]:
order_product_prior['product_seq_time'] = order_product_prior.groupby(['user_id','product_id'])['order_number'].rank()

<ipython-input-14-a4f470e58f8b>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  order_product_prior['product_seq_time'] = order_product_prior.groupby(['user_id','product_id'])['order_number'].rank()


In [15]:
m = order_product_prior.groupby('product_id')

In [16]:
prd_feature = pd.DataFrame({
    'prod_orders': m['order_number'].count(),
    'prod_reorders': m['reordered'].sum(),
    'prod_first_orders': order_product_prior[order_product_prior['product_seq_time'] == 1].groupby('product_id')['product_seq_time'].sum(),
    'prod_second_orders': order_product_prior[order_product_prior['product_seq_time'] == 2].groupby('product_id')['product_seq_time'].count()
})

In [17]:
prd_feature = prd_feature.reset_index()
prd_feature.head(10)

,product_id,prod_orders,prod_reorders,prod_first_orders,prod_second_orders
0,1,1852,1136,716.0,276.0
1,2,90,12,78.0,8.0
2,3,277,203,74.0,36.0
3,4,329,147,182.0,64.0
4,5,15,9,6.0,4.0
5,6,8,3,5.0,2.0
6,7,30,12,18.0,6.0
7,8,165,83,82.0,30.0
8,9,156,82,74.0,31.0
9,10,2572,1304,1268.0,399.0


## Initial Feature Engineering 2

In [29]:
data = pd.merge(user_feature_1, user_feature_2, on = 'user_id', how = 'left')
data = pd.merge(data, up_feature, on = 'user_id', how = 'left')
data = pd.merge(data, prd_feature, on = 'product_id', how = 'left')

In [30]:
data.head(10)

,user_id,user_orders,sum_days_since_prior_order,avg_days_since_prior_order,total_products,distinct_products,reordered_ratio,product_id,up_orders,up_first_order,up_last_order,up_average_cart_position,prod_orders,prod_reorders,prod_first_orders,prod_second_orders
0,1,10,176.0,17.6,59,18,0.759259,196,10,1,10,1.400000,35791,27791,8000.0,4660.0
1,1,10,176.0,17.6,59,18,0.759259,10258,9,2,10,3.333333,1946,1389,557.0,308.0
2,1,10,176.0,17.6,59,18,0.759259,10326,1,5,5,5.000000,5526,3603,1923.0,1003.0
3,1,10,176.0,17.6,59,18,0.759259,12427,10,1,10,3.300000,6476,4797,1679.0,889.0
4,1,10,176.0,17.6,59,18,0.759259,13032,3,2,10,6.333333,3751,2465,1286.0,617.0
5,1,10,176.0,17.6,59,18,0.759259,13176,2,2,5,6.000000,379450,315913,63537.0,45231.0
6,1,10,176.0,17.6,59,18,0.759259,14084,1,1,1,2.000000,15935,12923,3012.0,1895.0
7,1,10,176.0,17.6,59,18,0.759259,17122,1,5,5,6.000000,13880,9377,4503.0,2375.0
8,1,10,176.0,17.6,59,18,0.759259,25133,8,3,10,4.000000,6196,4586,1610.0,935.0
9,1,10,176.0,17.6,59,18,0.759259,26088,2,1,2,4.500000,2523,1360,1163.0,471.0


In [31]:
data['prod_reorder_probability'] = data.prod_second_orders / data.prod_first_orders
data['prod_reorder_times'] = 1 + data.prod_reorders / data.prod_first_orders
data['prod_reorder_ratio'] = data.prod_reorders / data.prod_orders
data.drop(['prod_reorders', 'prod_first_orders', 'prod_second_orders'], axis=1, inplace=True)
data['user_average_basket'] = data.total_products / data.user_orders
data['up_order_rate'] = data.up_orders / data.user_orders
data['up_orders_since_last_order'] = data.user_orders - data.up_last_order
data['up_order_rate_since_first_order'] = data.up_orders / (data.user_orders - data.up_first_order + 1)

In [34]:
prd = data['product_id']
data.drop(labels=['product_id'], axis=1,inplace = True)
data.insert(1, 'product_id', prd)
data.head(10)

,user_id,product_id,user_orders,sum_days_since_prior_order,avg_days_since_prior_order,total_products,distinct_products,reordered_ratio,up_orders,up_first_order,up_last_order,up_average_cart_position,prod_orders,prod_reorder_probability,prod_reorder_times,prod_reorder_ratio,user_average_basket,up_order_rate,up_orders_since_last_order,up_order_rate_since_first_order
0,1,196,10,176.0,17.6,59,18,0.759259,10,1,10,1.400000,35791,0.582500,4.473875,0.776480,5.9,1.0,0,1.000000
1,1,10258,10,176.0,17.6,59,18,0.759259,9,2,10,3.333333,1946,0.552962,3.493716,0.713772,5.9,0.9,0,1.000000
2,1,10326,10,176.0,17.6,59,18,0.759259,1,5,5,5.000000,5526,0.521581,2.873635,0.652009,5.9,0.1,5,0.166667
3,1,12427,10,176.0,17.6,59,18,0.759259,10,1,10,3.300000,6476,0.529482,3.857058,0.740735,5.9,1.0,0,1.000000
4,1,13032,10,176.0,17.6,59,18,0.759259,3,2,10,6.333333,3751,0.479782,2.916796,0.657158,5.9,0.3,0,0.333333
5,1,13176,10,176.0,17.6,59,18,0.759259,2,2,5,6.000000,379450,0.711884,5.972111,0.832555,5.9,0.2,5,0.222222
6,1,14084,10,176.0,17.6,59,18,0.759259,1,1,1,2.000000,15935,0.629150,5.290505,0.810982,5.9,0.1,9,0.100000
7,1,17122,10,176.0,17.6,59,18,0.759259,1,5,5,6.000000,13880,0.527426,3.082390,0.675576,5.9,0.1,5,0.166667
8,1,25133,10,176.0,17.6,59,18,0.759259,8,3,10,4.000000,6196,0.580745,3.848447,0.740155,5.9,0.8,0,1.000000
9,1,26088,10,176.0,17.6,59,18,0.759259,2,1,2,4.500000,2523,0.404987,2.169390,0.539041,5.9,0.2,8,0.200000


In [22]:
user_feature_1.shape

(206209, 4)

In [23]:
user_feature_2.shape

(206209, 4)

In [24]:
up_feature.shape

(13307953, 6)

In [25]:
prd_feature.shape

(49677, 5)